In [1]:
import os
from pprint import pprint as pp  # for pretty printing
from pystac_client import Client
import requests
import xarray
import rioxarray
import s3fs

# S3 keys, see tutorial for more details:
# https://documentation.dataspace.copernicus.eu/APIs/S3.html
S3_KEY = ""
S3_SECRET = ""

# Odata setup
# Your regular CDSE login credentials
# https://documentation.dataspace.copernicus.eu/APIs/OData.html
USER = ""
PASS = ""

# Section 1: STAC S3 Data Discovery

## General notes:

- The CDSE documentation is quite rubbish especially the STAC search: https://documentation.dataspace.copernicus.eu/APIs/STAC.html

- S3 Links are available in the STAC catalog.

- Accessing the S3 requires patience because you will get `forbidden` errors a lot. You need to retry over and over and it works.

- Sentinel-2 data appears to be merged in terms of `L1C` `L2A` and the documentation doesnt specify how to query one or the other. The data is in `.SAFE` as far as I can tell.

## Test 1: SENTINEL-2

### Summary

It is possible to extract the S3 Link from the STAC metadata as shown below, however:

1. Data is `L1C` and not `L2A` the CDSE docs don't make it clear, how to get the `L2A` data. It appears that SENTINEL-2 contains both so there needs to be some other filter option.

2. The Data is in `SAFE` format, which is not the most convenient format for processing.

In [2]:
catalog = Client.open('https://catalogue.dataspace.copernicus.eu/stac/')
items = catalog.search(
    collections=["SENTINEL-2"],
    bbox=[11.02083333, 46.65359938, 11.36666667, 46.95416667],
    datetime=['2022-02-01T00:00:00Z', '2022-06-30T00:00:00Z'],
).item_collection()

In [3]:
pp(items.items[0].get_links())

[<Link rel=root target=<Client id=Catalogue>>,
 <Link rel=self target=https://catalogue.dataspace.copernicus.eu/stac/collections/SENTINEL-2/items/S2B_MSIL1C_20220205T102119_N0400_R065_T32TPT_20220205T110638.SAFE>,
 <Link rel=collection target=https://catalogue.dataspace.copernicus.eu/stac/collections/SENTINEL-2>]


In [4]:
item_link = next(link for link in items.items[0].get_links() if link.rel == 'self').target
print(f"STAC Feature link: {item_link}")

STAC Feature link: https://catalogue.dataspace.copernicus.eu/stac/collections/SENTINEL-2/items/S2B_MSIL1C_20220205T102119_N0400_R065_T32TPT_20220205T110638.SAFE


In [5]:
item_response = requests.get(item_link)
item_details = item_response.json()
pp(item_details)

{'assets': {'PRODUCT': {'alternate': {'s3': {'href': '/eodata/Sentinel-2/MSI/L1C/2022/02/05/S2B_MSIL1C_20220205T102119_N0400_R065_T32TPT_20220205T110638.SAFE',
                                             'storage:platform': 'CLOUDFERRO',
                                             'storage:region': 'waw',
                                             'storage:requester_pays': False,
                                             'storage:tier': 'Online'}},
                        'href': 'https://catalogue.dataspace.copernicus.eu/odata/v1/Products(0164d3ee-55b6-559a-bdd9-3a80668253b1)/$value',
                        'title': 'Product',
                        'type': 'application/octet-stream'},
            'QUICKLOOK': {'href': 'https://catalogue.dataspace.copernicus.eu/odata/v1/Assets(45d156b7-0109-4766-b467-1a097bb97630)/$value',
                          'title': 'QUICKLOOK',
                          'type': 'image/jpeg'}},
 'bbox': [10.31188678741455,
          46.832672119140625

In [6]:
s3_path = item_details['assets']['PRODUCT']['alternate']['s3']['href']
s3_url = f"s3://{s3_path.lstrip('/')}"
print(f"S3 URL: {s3_url}")

S3 URL: s3://eodata/Sentinel-2/MSI/L1C/2022/02/05/S2B_MSIL1C_20220205T102119_N0400_R065_T32TPT_20220205T110638.SAFE


In [7]:
fs = s3fs.S3FileSystem(anon=False, key=S3_KEY, secret=S3_SECRET, client_kwargs={'endpoint_url': 'https://eodata.dataspace.copernicus.eu'})
def list_s3_objects(prefix):
    try:
        objects = fs.ls(prefix)
        for obj in objects:
            print(obj)
    except Exception as e:
        print(f"Failed to list objects in {prefix}: {e}")

print("\nListing objects in /eodata/Landsat-8/:")
list_s3_objects(s3_url)


Listing objects in /eodata/Landsat-8/:
eodata/Sentinel-2/MSI/L1C/2022/02/05/S2B_MSIL1C_20220205T102119_N0400_R065_T32TPT_20220205T110638.SAFE/
eodata/Sentinel-2/MSI/L1C/2022/02/05/S2B_MSIL1C_20220205T102119_N0400_R065_T32TPT_20220205T110638.SAFE/AUX_DATA
eodata/Sentinel-2/MSI/L1C/2022/02/05/S2B_MSIL1C_20220205T102119_N0400_R065_T32TPT_20220205T110638.SAFE/DATASTRIP
eodata/Sentinel-2/MSI/L1C/2022/02/05/S2B_MSIL1C_20220205T102119_N0400_R065_T32TPT_20220205T110638.SAFE/GRANULE
eodata/Sentinel-2/MSI/L1C/2022/02/05/S2B_MSIL1C_20220205T102119_N0400_R065_T32TPT_20220205T110638.SAFE/HTML
eodata/Sentinel-2/MSI/L1C/2022/02/05/S2B_MSIL1C_20220205T102119_N0400_R065_T32TPT_20220205T110638.SAFE/INSPIRE.xml
eodata/Sentinel-2/MSI/L1C/2022/02/05/S2B_MSIL1C_20220205T102119_N0400_R065_T32TPT_20220205T110638.SAFE/MTD_MSIL1C.xml
eodata/Sentinel-2/MSI/L1C/2022/02/05/S2B_MSIL1C_20220205T102119_N0400_R065_T32TPT_20220205T110638.SAFE/S2B_MSIL1C_20220205T102119_N0400_R065_T32TPT_20220205T110638-ql.jpg
eodata/S

## Test 2: LANDSAT-8

### Summary

Since LANDSAT 8 is `tif` we can select a random sample from the S3, load it and calculate NDVI. 



In [8]:
catalog = Client.open('https://catalogue.dataspace.copernicus.eu/stac/')
items = catalog.search(
    collections=["LANDSAT-8"],
    bbox=[11.02083333, 46.65359938, 11.36666667, 46.95416667],
    datetime=['2022-02-01T00:00:00Z', '2022-06-30T00:00:00Z'],
).item_collection()

In [9]:
pp(items.items[0].get_links())

item_link = next(link for link in items.items[0].get_links() if link.rel == 'self').target
print(f"STAC Feature link: {item_link}")

item_response = requests.get(item_link)
item_details = item_response.json()
pp(item_details)

s3_path = item_details['assets']['PRODUCT']['alternate']['s3']['href']
s3_url = f"s3://{s3_path.lstrip('/')}"
print(f"S3 URL: {s3_url}")

[<Link rel=root target=<Client id=Catalogue>>,
 <Link rel=self target=https://catalogue.dataspace.copernicus.eu/stac/collections/LANDSAT-8/items/LC08_L2SP_192028_20220201_20220205_02_T1>,
 <Link rel=collection target=https://catalogue.dataspace.copernicus.eu/stac/collections/LANDSAT-8>]
STAC Feature link: https://catalogue.dataspace.copernicus.eu/stac/collections/LANDSAT-8/items/LC08_L2SP_192028_20220201_20220205_02_T1
{'assets': {'PRODUCT': {'alternate': {'s3': {'href': '/eodata/Landsat-8/OLI_TIRS/L2SP/2022/02/01/LC08_L2SP_192028_20220201_20220205_02_T1',
                                             'storage:platform': 'CLOUDFERRO',
                                             'storage:region': 'waw',
                                             'storage:requester_pays': False,
                                             'storage:tier': 'Online'}},
                        'href': 'https://catalogue.dataspace.copernicus.eu/odata/v1/Products(bea5f1fd-ddb1-570b-a812-70b2d7e08973)/$value

In [10]:

fs = s3fs.S3FileSystem(anon=False, key=S3_KEY, secret=S3_SECRET, client_kwargs={'endpoint_url': 'https://eodata.dataspace.copernicus.eu'})
def list_s3_objects(prefix):
    try:
        objects = fs.ls(prefix)
        for obj in objects:
            print(obj)
    except Exception as e:
        print(f"Failed to list objects in {prefix}: {e}")

print("\nListing objects in /eodata/Landsat-8/:")
list_s3_objects(s3_url)


Listing objects in /eodata/Landsat-8/:
eodata/Landsat-8/OLI_TIRS/L2SP/2022/02/01/LC08_L2SP_192028_20220201_20220205_02_T1/
eodata/Landsat-8/OLI_TIRS/L2SP/2022/02/01/LC08_L2SP_192028_20220201_20220205_02_T1/LC08_L2SP_192028_20220201_20220205_02_T1_ANG.txt
eodata/Landsat-8/OLI_TIRS/L2SP/2022/02/01/LC08_L2SP_192028_20220201_20220205_02_T1/LC08_L2SP_192028_20220201_20220205_02_T1_MTL.json
eodata/Landsat-8/OLI_TIRS/L2SP/2022/02/01/LC08_L2SP_192028_20220201_20220205_02_T1/LC08_L2SP_192028_20220201_20220205_02_T1_MTL.txt
eodata/Landsat-8/OLI_TIRS/L2SP/2022/02/01/LC08_L2SP_192028_20220201_20220205_02_T1/LC08_L2SP_192028_20220201_20220205_02_T1_MTL.xml
eodata/Landsat-8/OLI_TIRS/L2SP/2022/02/01/LC08_L2SP_192028_20220201_20220205_02_T1/LC08_L2SP_192028_20220201_20220205_02_T1_QA_PIXEL.TIF
eodata/Landsat-8/OLI_TIRS/L2SP/2022/02/01/LC08_L2SP_192028_20220201_20220205_02_T1/LC08_L2SP_192028_20220201_20220205_02_T1_QA_RADSAT.TIF
eodata/Landsat-8/OLI_TIRS/L2SP/2022/02/01/LC08_L2SP_192028_20220201_2022

In [11]:
# Access the S3 data
# I dont take responsibility for the actual calculation of NDVI, it is just a showcase that it works
band4_path = 's3://eodata/Landsat-8/OLI_TIRS/L2SP/2022/02/01/LC08_L2SP_192028_20220201_20220205_02_T1/LC08_L2SP_192028_20220201_20220205_02_T1_SR_B4.TIF'
band5_path = 's3://eodata/Landsat-8/OLI_TIRS/L2SP/2022/02/01/LC08_L2SP_192028_20220201_20220205_02_T1/LC08_L2SP_192028_20220201_20220205_02_T1_SR_B5.TIF'

with fs.open(band4_path, 'rb') as f_band4:
    red_band = rioxarray.open_rasterio(f_band4)

with fs.open(band5_path, 'rb') as f_band5:
    nir_band = rioxarray.open_rasterio(f_band5)

nir_band = nir_band.rio.reproject_match(red_band)

ndvi = (nir_band - red_band) / (nir_band + red_band)

print(ndvi)

<xarray.DataArray (band: 1, y: 7751, x: 7641)> Size: 474MB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * band         (band) int64 8B 1
    spatial_ref  int64 8B 0
  * x            (x) float64 61kB 6.222e+05 6.222e+05 ... 8.514e+05 8.514e+05
  * y            (y) float64 62kB 5.218e+06 5.218e+06 ... 4.985e+06 4.985e+06


# Section 2: STAC Odata Data Discovery

## WIP

- Apart from an S3 link the STAC metadata also contains a link to the OData API
- I tried to make a sample work but I cant authorize for some reason

In [ ]:
catalog = Client.open('https://catalogue.dataspace.copernicus.eu/stac/')
items = catalog.search(
    collections=["SENTINEL-2"],
    bbox=[11.02083333, 46.65359938, 11.36666667, 46.95416667],
    datetime=['2022-02-01T00:00:00Z', '2022-06-30T00:00:00Z'],
).item_collection()


In [ ]:
pp(items.items[0].get_links())

item_link = next(link for link in items.items[0].get_links() if link.rel == 'self').target
print(f"STAC Feature link: {item_link}")

item_response = requests.get(item_link)
item_details = item_response.json()
pp(item_details)

odata_product_link = item_details['assets']['PRODUCT']['href']
print(f"OData Product link: {odata_product_link}")


In [ ]:
# Obtain the download token
token_url = 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token'
token_data = {
    'grant_type': 'password',
    'username': USER,
    'password': PASS,
    'client_id': 'cdse-public'
}
token_headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}

token_response = requests.post(token_url, data=token_data, headers=token_headers)

# Check if the request was successful
if token_response.status_code == 200:
    token_json = token_response.json()
    access_token = token_json.get('access_token')
    refresh_token = token_json.get('refresh_token')
    print(f"Access Token: {access_token}")
    print(f"Refresh Token: {refresh_token}")
else:
    print(f"Failed to obtain token. Status code: {token_response.status_code}")
    print(token_response.text) 
    access_token = None
    refresh_token = None

In [ ]:
headers = {
    'Authorization': f'Bearer {access_token}'
}

output_path = '/tmp/product.zip'
os.makedirs(os.path.dirname(output_path), exist_ok=True)

print(odata_product_link)

response = requests.get(odata_product_link, headers=headers, stream=True, allow_redirects=True)

if response.status_code == 200:
    with open(output_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print(f"File downloaded successfully and saved to {output_path}")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")
    print(response.text) 